In [1]:
import time
from datetime import timedelta

import html
import re

import os
import ntpath

import subprocess as sp

import numpy as np
import pandas as pd
import xml.etree.ElementTree as et


## Enter full path to CSV file

In [2]:
## Enter full path to CSV file

#common_path = '../../../../stackexchange_v2/workspace/input'
common_path = '../SO_data_dump'

## Enter Relative Path to csv file
relative_path = 'javarawcodes_csv/JavaRawCodes0386.csv'

filepath_javaanswers = '{}/{}'.format(common_path, relative_path)

## Read Java Raw Codes csv files 

In [3]:
ddf_javarawcodes = pd.read_csv(filepath_javaanswers, engine='python', error_bad_lines=False, warn_bad_lines=False, dtype=object)

In [4]:
# Keep a copy
javarawcodes_temp_df = ddf_javarawcodes.copy()

## Numbere of Posts Containing Code Tags

In [5]:
# Check the length of the dataframe WITHOUT dropped records
#javarawcodes_len = len(ddf_javarawcodes.index)
#javarawcodes_len

## Drop all row with any column having a NaN value

In [6]:
# Drop all row with any column having a NaN value
#ddf_javarawcodes = ddf_javarawcodes.dropna()
ddf_javarawcodes = ddf_javarawcodes[~ddf_javarawcodes.Code.isna()] # Drop rows that have NaN in the Code column

In [7]:
# Get all the dropped rows with NA
javarawcodes_na_df = javarawcodes_temp_df[~javarawcodes_temp_df.Idx.isin(ddf_javarawcodes.Idx)]

In [8]:
# Number of dropped records
javarawcodes_na_len = javarawcodes_na_df.index.shape[0]
javarawcodes_na_len

0

In [9]:
# Number of records left
javarawcodes_notna_len = ddf_javarawcodes.index.shape[0]
javarawcodes_notna_len

624

## Remove Codes that have any form of html or xml tags

#### Convert all the unicode strings into actual symbol

In [10]:
#https://stackoverflow.com/questions/60088353/convert-html-characters-to-strings-in-pandas-dataframe
ddf_javarawcodes = ddf_javarawcodes.applymap(html.unescape)

In [11]:
ddf_javarawcodes.index.shape[0]

624

In [12]:
# keep a copy
javarawcodes_temp_df = ddf_javarawcodes.copy()

In [13]:
#@Anyi some rows contain null
ddf_javarawcodes = ddf_javarawcodes.dropna(how='any',axis=0) 

In [14]:
ddf_javarawcodes.index.shape[0]

624

In [15]:
#@Anyi some rows contain null string
ddf_javarawcodes = ddf_javarawcodes[ddf_javarawcodes["Code"].str.contains("null", regex=True)==False]

In [16]:
ddf_javarawcodes.reset_index(inplace=True)

In [17]:
ddf_javarawcodes.index.shape[0]

600

In [18]:
#@Anyi some rows contain null string
#ddf_javarawcodes = ddf_javarawcodes[ddf_javarawcodes["Code"].str.contains("((.|\n|\r\n)*)?null", regex=True)==False]

In [19]:
ddf_javarawcodes.index.shape[0]

600

In [20]:
for row in ddf_javarawcodes["Idx"].to_list():
    if row == "12133763_12133655_96":
        print(row)

12133763_12133655_96
12133763_12133655_96
12133763_12133655_96
12133763_12133655_96


In [21]:
#itertool.zip_longest(list1, list2)
for row_col1, row_col2 in zip(ddf_javarawcodes["Idx"].to_list(), ddf_javarawcodes["Code"].to_list()):
    if row_col1 == "12133763_12133655_96":
        print(row_col1, row_col2)

12133763_12133655_96 WhileStatement:
    while ( Expression ) Statement

12133763_12133655_96 Expression:
    AssignmentExpression

12133763_12133655_96 AssignmentExpression:
    ConditionalExpression
    Assignment

Assignment:
    LeftHandSide AssignmentOperator AssignmentExpression

LeftHandSide:
    ExpressionName
    FieldAccess
    ArrayAccess

12133763_12133655_96 LeftHandSide


In [22]:
count = 0
for row in ddf_javarawcodes["Code"].to_list():
    count=count+1
print(count)

600


In [23]:
#for i in range(len(ddf_javarawcodes)):
     #print(ddf_javarawcodes["Idx"][i])
    #if ddf_javarawcodes.loc[i, "Idx"] == "12133760_12133655_95":
        #print(ddf_javarawcodes.loc[i, "Idx"], ddf_javarawcodes.loc[i, "Code"])
       

In [24]:
ddf_javarawcodes["Code"].str.contains('<.*>(.|\n|\r\n)<\/.*>*?')

/opt/nesi/CS400_centos7_bdw/Python/3.9.5-gimkl-2020a/lib/python3.9/site-packages/pandas/core/strings/accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


0      False
1      False
2      False
3      False
4      False
       ...  
595    False
596     True
597     True
598    False
599    False
Name: Code, Length: 600, dtype: bool

#### Remove code with any form of HTML/XML

In [25]:
#Replace all the single line html tags <.../> with empty string
#javarawcodes_html_df1 = ddf_javarawcodes[ddf_javarawcodes.Code.str.contains('^(//.*|\'|\n|\r|\s|\r\n)*<(.|\n|\r\n)*?>(.|\n|\r\n)*?<\/(.|\n|\r\n)*?(>|\'|\n|\r\n)$', regex=True)]
#ddf_javarawcodes_htmlxml = ddf_javarawcodes[ddf_javarawcodes.Code.str.contains('<.*>(.|\n|\r\n)*?<\/.*>', regex=True, na=False)]
ddf_javarawcodes_htmlxml = ddf_javarawcodes[ddf_javarawcodes.Code.str.contains('<.*>(.|\n|\r\n)<\/.*>*?', regex=True, na=False)]

#ddf_javarawcodes_htmlxml =ddf_javarawcodes[ddf_javarawcodes.map_partitions(lambda df : df.Code.str.contains('<.*>(.|\n|\r\n)*?<\/.*>', regex=True))]

In [26]:
ddf_javarawcodes_htmlxml.head()

,index,Idx,match,Code
169,179,12134913_12130723_157,16,CREATE TABLE some_table (id integer primary ke...
476,502,12138235_12137892_320,0,"<activity\n android:name="".Activityone""..."
477,503,12138287_12090406_321,0,"<bean id=""dataSource"" class=""someProject.DbCon..."
525,551,12138635_12138537_341,0,<context-param>\n<param-name>contextConfigLoca...
550,576,12138989_12138905_350,0,<error-page>\n <error-code>404</error-code>...


In [27]:
# Number of records that dropped
#javarawcodes_htmlxml_len = ddf_javarawcodes_htmlxml.index.shape[0].compute()
javarawcodes_htmlxml_len = ddf_javarawcodes_htmlxml.Idx.shape[0].compute()
javarawcodes_htmlxml_len

AttributeError: 'int' object has no attribute 'compute'

In [ ]:
# Get all the records that DO NOT contains html tags regex
ddf_javarawcodes = ddf_javarawcodes[~ddf_javarawcodes.Idx.isin(ddf_javarawcodes_htmlxml.Idx.compute())]

In [ ]:
# Number of records left
javarawcodes_len = ddf_javarawcodes.index.shape[0].compute()
javarawcodes_len

In [ ]:
#Verify lengths
#javarawcodes_len + javarawcodes_htmlxml_len

## Drop java code with one line of code

In [ ]:
# Keep a copy
javarawcodes_temp_df = ddf_javarawcodes.copy()

In [ ]:
# Get df contains more than one line of java code
ddf_javarawcodes = ddf_javarawcodes[ddf_javarawcodes.Code.str.contains('\n')]

In [ ]:
# Number of rows that contains more than one line of java code
javarawcodes_len = ddf_javarawcodes.index.shape[0].compute()
javarawcodes_len

In [ ]:
# Get the dataframe that contains one line of java code
javarawcodes_oneline_df = javarawcodes_temp_df[~javarawcodes_temp_df.Idx.isin(ddf_javarawcodes.Idx.compute())]

In [ ]:
# Number of rows that contains one line of java code
javarawcodes_oneline_len = javarawcodes_oneline_df.index.shape[0].compute()
javarawcodes_oneline_len

In [ ]:
# Verify that the lengths are equal to the javarawcodes_html_len1
javarawcodes_len + javarawcodes_oneline_len

## Get complete java code snippet

In [ ]:
# Get the rows that contain import, package or class
# using regex to search for text that contain import, package or class
ddf_complete_javacodes = ddf_javarawcodes[(ddf_javarawcodes['Code'].str.contains('^import\s+\w+(\.+\w+)*', case=False, regex=True))|
         (ddf_javarawcodes['Code'].str.contains('^package\s+\w+(\.+\w+)*', case=False, regex=True))|
         (ddf_javarawcodes['Code'].str.contains('(class|interface)(\s+\w+)(\s+\w+)*(\s*{)', case=False, regex=True))]

In [ ]:
# Check lenght of complete java code snippet
complete_javacodes_len = ddf_complete_javacodes.index.shape[0].compute()
complete_javacodes_len

## Get incomplete java code snippet

In [ ]:
# Get all rows not contained in ddf_complete_javacodes
ddf_incomplete_javacodes = ddf_javarawcodes[~ddf_javarawcodes.Idx.isin(ddf_complete_javacodes.Idx.compute())]

In [ ]:
# Check lenght of incomplete java code snippet
ddf_incomplete_javacodes.index.shape[0].compute()

In [ ]:
# Total lenght of both complete and incomplete equals javarawcodes_len
ddf_complete_javacodes.index.shape[0].compute() + ddf_incomplete_javacodes.index.shape[0].compute()

## Furthermore drop codes that starts with $, WARNING:, >java, java, >javac, javac or contain some build, src directories

In [ ]:
# Furthermore drop codes that starts with $, WARNING:, >java, java, >javac, javac, <xml and html tags> and directories

ddf_non_codes = ddf_incomplete_javacodes[
    (ddf_incomplete_javacodes.Code.str.contains('^\$.*', case=False, regex=True))|
    (ddf_incomplete_javacodes.Code.str.contains('^WARNING:(\s*\w*)*', case=False, regex=True))|
    (ddf_incomplete_javacodes.Code.str.contains('^($|>|java|javac|jar|javacc)(\s*\w*\.*)*', case=False, regex=True))|
    (ddf_incomplete_javacodes.Code.str.contains('(\\n)*\s*\.+\/+.*\s*\.*\/*(build|src)', case=False, regex=True))
]

In [ ]:
# Check lenght of the non codes java code snippet 
non_codes_len = ddf_non_codes.index.shape[0].compute()
non_codes_len

In [ ]:
# get all the dropped rows that starts with $, WARNING:, >java, java, >javac, javac or directory
ddf_incomplete_javacodes = ddf_incomplete_javacodes[~ddf_incomplete_javacodes.Idx.isin(ddf_non_codes.Idx.compute())]

In [ ]:
# Check lenght of incomplete java code snippet after html tags are dropped
incomplete_javacodes_len = ddf_incomplete_javacodes.index.shape[0].compute()
incomplete_javacodes_len

In [ ]:
# Verify that the length is equals to combination of complete_javarawcodes_len and incomplete_javarawcodes_len above
incomplete_javacodes_len + non_codes_len

In [ ]:
nnn

### Make a complete codes folder in that directory

In [ ]:
## Make a folder in that directory
folder = '{}/javacompletecodesnippets_csv'.format(common_path)
# output: path/to/Post.csv => path/to
mkdir_cmd = 'mkdir {}'.format(folder)
cmd = sp.run(
    mkdir_cmd, # command
    capture_output=True,
    text=True,
    shell=True
)

In [ ]:
print(folder)

### Save Java complete codes csv files in that directory

In [ ]:
## Save files in that directory
filename = 'JavaCompleteCodeSnippets'
file = '{}/{}*.csv'.format(folder, filename)
_ = ddf_complete_javacodes.to_csv(file, sep=',', index=False)

### Make a incomplete codes folder in that directory

In [ ]:
## Make a folder in that directory
folder = '{}/javaincompletecodesnippets_csv'.format(common_path)
# output: path/to/Post.csv => path/to
mkdir_cmd = 'mkdir {}'.format(folder)
cmd = sp.run(
    mkdir_cmd, # command
    capture_output=True,
    text=True,
    shell=True
)

In [ ]:
print(folder)

### Save Java incomplete codes csv files in that directory

In [ ]:
## Save files in that directory
filename = 'JavaIncompleteCodeSnippets'
file = '{}/{}*.csv'.format(folder, filename)
_ = ddf_incomplete_javacodes.to_csv(file, sep=',', index=False)

### Make a code that contains HTML/XML folder in that directory 

In [ ]:
## Make a folder in that directory
folder = '{}/javahtmlxmltags_csv'.format(common_path)
# output: path/to/Post.csv => path/to
mkdir_cmd = 'mkdir {}'.format(folder)
cmd = sp.run(
    mkdir_cmd, # command
    capture_output=True,
    text=True,
    shell=True
)

### Save Java codes that contains HTML/XML csv files in that directory

In [ ]:
## Save files in that directory
filename = 'JavaCodeHtmlXmlTags'
file = '{}/{}*.csv'.format(folder, filename)
_ = ddf_javarawcodes_htmlxml.to_csv(file, sep=',', index=False)

### Make a noncode folder in that directory 

In [ ]:
## Make a folder in that directory
folder = '{}/noncode_csv'.format(common_path)
# output: path/to/Post.csv => path/to
mkdir_cmd = 'mkdir {}'.format(folder)
cmd = sp.run(
    mkdir_cmd, # command
    capture_output=True,
    text=True,
    shell=True
)

In [ ]:
print(folder)

### Save non codes csv files in that directory

In [ ]:
## Save files in that directory
filename = 'NonCodes'
file = '{}/{}*.csv'.format(folder, filename)
_ = ddf_non_codes.to_csv(file, sep=',', index=False)

## Replace all the interface and class names to all have a common name (e.g. Code) in the ddf_complete_javacodes

In [ ]:
# Replace all the class with the class to all have a common name, say Code
ddf_complete_javacodes = ddf_complete_javacodes.replace(to_replace = 'class\s+\w+', value = 'class Code', regex=True)

In [ ]:
# Also Replace interface with the interface Code
ddf_complete_javacodes = ddf_complete_javacodes.replace(to_replace = 'interface\s+\w+', value = 'interface Code', regex=True)

## Encapsulate incomplete written java program with public Code { ... }

In [ ]:
# Copy the code to Code2
#ddf_incomplete_javacodes['Code2'] = ddf_incomplete_javacodes['Code']

In [ ]:
# Encapsulate incomplete written java program with public Code {  }
ddf_incomplete_javacodes['Code'] = 'public class Code {\n'+ ddf_incomplete_javacodes['Code'].astype(str) +'\n}'

In [ ]:
ddf_complete_javacodes.columns

In [ ]:
ddf_incomplete_javacodes.columns

## Merge the data frame containing the completed java codes with the data frame containing the incomplete code (encapsulated with public Code {})

In [ ]:
# merge complete and incomplete into one dataframe
df = dd.concat([ddf_complete_javacodes, ddf_incomplete_javacodes])

In [ ]:
df..index.shape[0].compute()

In [ ]:
# Verify that the combination of the lengths is same as merged len
complete_javacodes_len + incomplete_javacodes_len

In [ ]:
df.columns

## Replace all cases where three dots ... appeared at the begining of a line within the code with comment i.e. //...

In [ ]:
df['Code'] = df.Code.replace(to_replace ='(\\n)+\s*\.{2,}', value = '\n\n//...\n\n', regex=True)
df['Code'] = df.Code.replace(to_replace ='(\\n)+\s*\.{2,}', value = '\n\n//...\n\n', regex=True)
df['Code'] = df.Code.replace(to_replace ='(\\n\.)+', value = '\n\n//...\n\n', regex=True)
df['Code'] = df.Code.replace(to_replace ='\{(\\n)+\s*\.{2,}', value = '{\n\n//...\n\n', regex=True)
df['Code'] = df.Code.replace(to_replace ='\[(\\n)+\s*\.{2,}', value = '[\n\n//...\n\n', regex=True)
df['Code'] = df.Code.replace(to_replace ='\{\s*\.{2,}', value = '{\n\n//...\n\n', regex=True)
df['Code'] = df.Code.replace(to_replace ='\[\s*\.{2,}', value = '[\n\n//...\n\n', regex=True)

#### To verify if the Regex works

In [ ]:
#@@ This is supposed tp capture anywhere 2 or more dots occured anywhere in the code 
doted_df1 = df[df.Code.str.contains('((\.\s){2,}|\.{2,})', case=False, regex=True)] #works

In [ ]:
doted_df1.index.shape[0].compute()

In [ ]:
#this is ment to capture anywhere we have three dots without comment //
doted_df2 = doted_df1[doted_df1.Code.str.contains('((\[|{)(\\n)*\s*|(\\n)+\s*)\.{2,}|(\\n\.)+', case=False, regex=True)]

In [ ]:
doted_df2.index.shape[0].compute()

In [ ]:
doted_df3=doted_df1[~doted_df1.Idx.isin(doted_df2.Idx.compute())]

In [ ]:
doted_df3.index.shape[0].compute()

## Uniquely identify each class name by appending the index_questionid to the end of the class name

In [ ]:
# To uniquely identify each class, append the index number of each row to Classname

# e.g. 
# Code_44671882_44671797_3688_0 
# Code_44671882_44671797_3688_2
# Code_44671809_32500182_1042_0
# ...

# Explanation:
# e.g. Code0_44671882_44671797_3688_0
# CodeUniqueIndex_PostID_ParentID_GroupID_MatchCaseNumber
# Code0_44671882_44671797_3688_1 did not appear becous it could have been removed while preprocessing

# Importing re package for using regular expressions 
import re 

#def appendIndexToClassNames(index, questionIdx, match, javacode): 
def appendIndexToClassNames(questionIdx, match, javacode): 
    
    # Search for class ClassName 
    # i.e. class then any characters repeated any number of times 
    if re.search('(class|interface)\s+\w+', javacode): 
  
        # Extract the position of end of pattern 
        #start = re.search('(class|interface)\s+\w+', javacode).start()
        end = re.search('(class|interface)\s+\w+', javacode).end()
  
        # return the cleaned name 
        # return javacode[ : pos]+str(javacode.index)+javacode[pos:]
        return javacode[ : end]+'_'+str(questionIdx)+'_'+str(match)+javacode[end: ]
  
    else: 
        # if clean up needed return the same name 
        return javacode 
          
 

In [ ]:
# Updated the Java Code answers column  
df['Code'] = df.apply(
         #lambda row: appendIndexToClassNames(row.name, row['Idx'], row['match'], row['Code']),axis=1)
    lambda row: appendIndexToClassNames(row['Idx'], row['match'], row['Code']),
         axis=1)
# Print the updated dataframe 
#print(df)

### Make a folder in that directory

In [ ]:
## Make a folder in that directory
folder = '{}/codesnippets_csv'.format(common_path)
# output: path/to/Post.csv => path/to
mkdir_cmd = 'mkdir {}'.format(folder)
cmd = sp.run(
    mkdir_cmd, # command
    capture_output=True,
    text=True,
    shell=True
)

In [ ]:
print(folder)

### Save files in that directory

In [ ]:
## Save files in that directory
filename = 'JavaCodeSnippets'
file = '{}/{}*.csv'.format(folder, filename)
_ = df.to_csv(file, sep=',', index=False)